# Einführung in die Polynomiale Regression
### Es geht um landwirtschftliche Flurstücke und deren Verkauspreise: Kann man den Verkaufspreis vorhersagen, wenn man Länge und Breite der Grundstücke kennt?

In [1]:
import pandas as pd

df = pd.read_csv("../fields.csv")

In [2]:
df.head(30)

,length,profit,width
0,807.0,634630.0,1032.0
1,299.0,124074.0,337.0
2,431.0,1338300.0,1631.0
3,744.0,327720.0,553.0
4,364.0,500244.0,827.0
5,977.0,1284950.0,1608.0
6,969.0,96045.0,207.0
7,339.0,868313.0,1208.0
8,443.0,1471693.0,1723.0
9,488.0,156123.0,392.0


In [3]:
# Beispiel: Normale, lineare Regression

X = df[["width", "length"]].values
Y = df[["profit"]].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state = 42, test_size = 0.25)

from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)

print(model.score(X_test, y_test))

0.9151374097035714


In [4]:
from sklearn.preprocessing import PolynomialFeatures

PolynomialFeatures?


Init signature:
PolynomialFeatures(
    degree=2,
    *,
    interaction_only=False,
    include_bias=True,
    order='C',
)
Docstring:     
Generate polynomial and interaction features.

Generate a new feature matrix consisting of all polynomial combinations
of the features with degree less than or equal to the specified degree.
For example, if an input sample is two dimensional and of the form
[a, b], the degree-2 polynomial features are [1, a, b, a^2, ab, b^2].

Read more in the :ref:`User Guide <polynomial_features>`.

Parameters
----------
degree : int or tuple (min_degree, max_degree), default=2
    If a single int is given, it specifies the maximal degree of the
    polynomial features. If a tuple `(min_degree, max_degree)` is passed,
    then `min_degree` is the minimum and `max_degree` is the maximum
    polynomial degree of the generated features. Note that `min_degree=0`
    and `min_degree=1` are equivalent as outputting the degree zero term is
    determined by `include_bi

In [ ]:
pf = PolynomialFeatures(degree = 2, include_bias = False)
#include_bias = False  - Wenn das auf True ist, dann gibt es eine zusätzliche Spalte mit einer 1, um den Intercept zu trainieren, falls diser nicht vorhanden wäre.

pf.fit(X_train) #wäre hier strenggenommen nicht nötig, aber andere Prozesse verlangen das später


In [ ]:
X_train
#Ein Arry aus Paaren "length" und "width" bzw. wir sagen x1 und x2

In [ ]:
pf.transform(X_train)
#erzeugt  jetzt 5 Zahlen pro Example und zwar: x1, x2, x1*x1, X1*x2,  x2*x2   

In [ ]:
#der Bauplan hierfür ist mit
pf.powers_
#abrufbar. Dort sind die Potenzen der 5 Spalten sichtbar. 
# width ^ 1 * length ^ 0
# width ^ 2 * length ^ 0

In [ ]:
#pf.transform(X_train)[:, [0, 2]]
# schneidet z.B. X1, X1*X1 heraus... (nur Beispiel, das zeigen soll, wie man Spalten herausschneidet)

In [ ]:
# Durch manuelles trial-and-error (Herumprobieren), kann man herausfinden, was hier wirklich
# ausschlaggebend ist, um auf R2 = 0,988 zu kommen:

#X_train_transformed = pf.transform(X_train) # Alles ohne Filter - liefert 0,988 - Bravo!
#X_test_transformed = pf.transform(X_test) # Alles ohne Filter - liefert 0,988 - Bravo!
#X_train_transformed = pf.transform(X_train)[:, [0, 1, 2, 3, 4]] # Komplett alles nehmen
#X_test_transformed = pf.transform(X_test)[:, [0, 1, 2, 3, 4]] # ...ist hier nicht falsch.
#X_train_transformed = pf.transform(X_train)[:, [0, 2]] # liefert auch 0,988
#X_test_transformed = pf.transform(X_test)[:, [0, 2]]   # und das entspricht der Transformation: Länge, Länge * Länge
#X_train_transformed = pf.transform(X_train)[:, [3]] # entspricht der Transformation: Länge * Breite, was den Preis gut repräsentieren sollte,
#X_test_transformed = pf.transform(X_test)[:, [3]]   # aber das bingt nur 0,53 und ist eigentlich gegen die Intuition.
#X_train_transformed = pf.transform(X_train)[:, [0, 3]] # Das entspricht Länge, Länge * Breite,
#X_test_transformed = pf.transform(X_test)[:, [0, 3]]  # aber das bingt gegen die Intuition komischerweise nicht die besten Ergebnisse

X_train_transformed = pf.transform(X_train)[:, [1,2]]  # Breite und Länge^2 ...funktionieren gut!
X_test_transformed = pf.transform(X_test)[:, [1,2]]

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train_transformed, y_train)

print(model.score(X_test_transformed, y_test))

In [ ]:
model.coef_

In [ ]:
model.intercept_

In [ ]:
#Was passiert eigentlich bei pf.transform genau
from sklearn.preprocessing import PolynomialFeatures
pf = PolynomialFeatures(degree = 4, include_bias = False)
a = [[1,2,3,4],
 [2,10,3,4],
 [2,11,3,4],
 [2,12,3,4]]
pf.fit(a)
pf.transform(a)

In [ ]:
pf.powers_

In [ ]:
# 1. Zeile - gegeben: 1 2 3 4 
# 1 2 3 4
# 1*1  1*2 1*3 1*4
# 2*2  2*3  2*4
# 3*3 3*4
# 4*4

# 2. Zeile - gegeben: 2 10 3 4
# 2 10 3 4
# 2*2 2*10 2*3 2*4
# 10*10 10*30 10*4
# 3*3 3*4
# 4*4

# usw. usw.

In [ ]:
pf.transform(a)[:, [0, 5]]